In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [28]:
# Cargar los datos desde el archivo CSV
data = pd.read_csv("./data/etecsa_processed.csv")

In [32]:
# Ordenar los datos por usuario, día y tiempo de inicio
data = data.sort_values(by=['user_index', 'day', 'time_start'])

# Agrupar por usuario y día
grouped = data.groupby(['user_index', 'day'])

# Función para procesar cada grupo
def process_group(group):
    zones = group['zone'].to_numpy()
    time_start = group['time_start'].to_numpy()
    time_end = group['time_end'].to_numpy()
    
    t_time = time_end[-1] - time_start[0]
    
    # Cálculo vectorizado para f_time
    f_time = (time_start[1:-1] - time_start[0]) / t_time

    # Crear DataFrame para las trayectorias procesadas
    processed = pd.DataFrame({
        'zone_start': zones[0],
        'zone_middle': zones[1:-1],
        'zone_end': zones[-1],
        'f_time': f_time
    })
    return processed

# Aplicar la función a cada grupo y concatenar los resultados
processed_trajectories = grouped.apply(process_group)

# Eliminar el índice adicional generado por el groupby
processed_trajectories.reset_index(drop=True, inplace=True)

# Guardar los datos procesados en un nuevo archivo CSV
processed_trajectories.to_csv("./data/processed_trajectories.csv", index=False)

C:\Users\alexs\AppData\Local\Temp\ipykernel_133060\3256916026.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_trajectories = grouped.apply(process_group)


In [ ]:
# Cargar el archivo shapefile
shp_file = 'data/havana_shape/havana_transport_zones.shp'
data = gpd.read_file(shp_file)

# Calcular los centroides
data['centroid'] = data.geometry.centroid

# Extraer las coordenadas (latitud y longitud)
data['latitude'] = data['centroid'].y
data['longitude'] = data['centroid'].x

zones_centroids = data[['No_de_zona', 'latitude', 'longitude']].rename(columns={'No_de_zona': 'zone'})

# Mostrar los primeros registros
zones_centroids.head()

zones_centroids.to_csv("./data/zones_centroids.csv", index=False)